In [1]:
import subprocess
import json
import os
from constant import *
from dotenv import load_dotenv
from web3 import Web3
from eth_account import Account
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from bit import Key
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy

In [2]:
mnemonic = os.getenv('MNEMONIC')
print(mnemonic)

None


In [3]:
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [4]:
# Derive Addresses

def derive_wallets(mnemoic, coin, numderive):
    command = f' ./hd-wallet-derive.php -g --mnemonic="{mnemonic}" --cols=path,address,privkey,pubkey --coin={coin} --numderive={numderive} --format=json'
    
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    
    keys = json.loads(output)
    print(keys)

In [5]:
# Retrieve key

coins = {"eth":derive_wallets(mnemonic, ETH, 3), "btc-test":derive_wallets(mnemonic, BTCTEST, 3)}
numderive = 3
print(coins["eth"][2]['privkey'])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [16]:
# Retrieve Keys

keys = {}
for coin in coins:
    keys[coin]= derive_wallets(mnemonic, coin, numderive=3)

NameError: name 'coins' is not defined

In [ ]:
# Retrieve Keys

eth_PrivateKey = keys["eth"][0]['privkey']
btc_PrivateKey = keys['btc-test'][0]['privkey']
print(json.dumps(eth_PrivateKey, indent=4, sort_keys=True))
print(json.dumps(btc_PrivateKey, indent=4, sort_keys=True))
print(json.dumps(keys, indent=4, sort_keys=True))

In [ ]:
# Next Cells show keys being retrieved through shell

In [ ]:
#get eth wallets ./hd-wallet-derive.php -g --mnemonic="machine powder duty spot price cook either bracket degree limit sword split" --cols=path,address,privkey,pubkey --coin=eth --numderive=3 --format=json

#eth results [{"path":"m\/44'\/60'\/0'\/0\/0","address":"0x5C9C1F0D4bE96F0f38f47640f3C697Db2492A513",
#"privkey":"0xa402b50dca62e364c5f2aaa231115a1bedd90ce0a5d7e64cea118ca23fea203b",
#"pubkey":"03f111f74903fe5908caa4bc79371a9f22af09349743f48a815be484ba6e6cc0a5"},
#{"path":"m\/44'\/60'\/0'\/0\/1","address":"0x6E4b8B9e713a5FE83027BBd469d8c2865a81C0bf",
#"privkey":"0x8b982802ac4dbc920e5e665d4160d229bbdc4de634ee2afd62a53ebdb1d4ecea",
#"pubkey":"02dfc13958424e03809c0b3f4bdb726e960f1013b7551a9375669ade7cf74bad90"},
#{"path":"m\/44'\/60'\/0'\/0\/2","address":"0x263Dafb59dBaC0b39BB2FFF541D05035650ec9A4",
#"privkey":"0x60af1af06a2de39cbcb5d2aa5aa04172b4adae30e58484d76dd1ff55d00c6308",
#"pubkey":"0286d4f6288812086b73fa42eadcb7429d59838db2e8716a0f801bec7fbc14eb04"}](crypto)


In [ ]:
#get btc-test wallets ./hd-wallet-derive.php -g --mnemonic="machine powder duty spot price cook either bracket degree limit sword split" --cols=path,address,privkey,pubkey --coin=BTC-test --numderive=3 --format=json

# result [{"path":"m\/44'\/1'\/0'\/0\/0","address":"miwuC7ugJZRZcvRo7TzfBgAAUggh8BuRMf",
#"privkey":"cNud17LHb5sXZRsYmCYP25asoprNGBFQVLFqwXvJ8bjZUsrivdHC",
#"pubkey":"02c52fa107d51405e730c15a3b64850eac48b3610955514c545e3b1f5fe93a334b"},
#{"path":"m\/44'\/1'\/0'\/0\/1","address":"mn39ger5Le2u8d2BcSKxSbaJziSQ6K5wnS",
#"privkey":"cPFJJvMwDLA1NAhHpgmMaUFFcJU4JHB9rpfjJrR5xmoqhJ3iZsWf",
#"pubkey":"03c7c0d130b4f2c7ffacbad1735fb697784c45acb882039f1861ec6d31882f0c8e"},
#{"path":"m\/44'\/1'\/0'\/0\/2","address":"n1aCSAbC8LuWNDcoGoAn1NGwk2jHuzKoCL",
#"privkey":"cVDJB5ucY6B9grQvs8bVbREyMwMddoTM96yn6SS8XkVQrC7bJsPD",
#"pubkey":"034e3e9a2433bb01e14e18e8799b3a8b940e558fd3d2cd32fe5b1ca38127a781b0"}](crypto)


In [38]:
#Set up private key
def priv_key_to_account(coin,priv_key):
    print(coin)
    print(priv_key)
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [35]:
#Prepare Transaction
def create_tx(coin,account, recipient, amount):
    if coin == ETH: 
        gasEstimate = w3.eth.estimateGas(
            {"from":eth_acc.address, "to":recipient, "value": amount}
        )
        return { 
            "from": eth_acc.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(eth_acc.address)
        }
    
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [36]:
#Send Transaction
def send_txn(coin,account,recipient, amount):
    txn = create_tx(coin, account, recipient, amount)
    if coin == ETH:
        signed_txn = eth_acc.sign_transaction(txn)
        result = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
        print(result.hex())
        return result.hex()
    elif coin == BTCTEST:
        tx_btctest = create_tx(coin, account, recipient, amount)
        signed_txn = account.sign_transaction(txn)
        print(signed_txn)
        return NetworkAPI.broadcast_tx_testnet(signed_txn)


In [40]:
#BTC Transaction
sending_btc_key = 'cNud17LHb5sXZRsYmCYP25asoprNGBFQVLFqwXvJ8bjZUsrivdHC'
btc_acc = priv_key_to_account(BTCTEST,sending_btc_key)

create_tx(BTCTEST,btc_acc,"miwuC7ugJZRZcvRo7TzfBgAAUggh8BuRMf", 0.003)
send_txn(BTCTEST,btc_acc,"mn39ger5Le2u8d2BcSKxSbaJziSQ6K5wnS", 0.003)

btc-test
cNud17LHb5sXZRsYmCYP25asoprNGBFQVLFqwXvJ8bjZUsrivdHC
01000000018729b68c8225a04d41b7f31464b628c799f502c3344701db49e9f396959e926d000000006a4730440220619ff0e91dd82012dcaaecfc4b2e852f49da25f31382e973927b1fc95debf42902203f5da235c4928bedf6990095fe39f4f1b76821192addbb98dda9d91280f152c9012102c52fa107d51405e730c15a3b64850eac48b3610955514c545e3b1f5fe93a334bffffffff02e0930400000000001976a9144786bdbc2033f494519b135cb9a277a7f8c9648b88ac75341900000000001976a91425a02fcca589f35a785e9ab3cda755d2cbc95a7288ac00000000


In [41]:
#Check BTC Test Balance
test_trans_key = PrivateKeyTestnet('cPFJJvMwDLA1NAhHpgmMaUFFcJU4JHB9rpfjJrR5xmoqhJ3iZsWf')
test_trans_key.get_balance('btc')

'0.003'

In [21]:
#Connect to local Network
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545/0xa402b50dca62e364c5f2aaa231115a1bedd90ce0a5d7e64cea118ca23fea203b"))

In [22]:
w3.isConnected()

True

In [42]:
#Sending Eth

eth_acc = priv_key_to_account(ETH, "0xa402b50dca62e364c5f2aaa231115a1bedd90ce0a5d7e64cea118ca23fea203b")

create_tx(ETH,eth_acc,"0x5C9C1F0D4bE96F0f38f47640f3C697Db2492A513", 1000)
send_txn(ETH, eth_acc,"0x263Dafb59dBaC0b39BB2FFF541D05035650ec9A4", 1000)

eth
0xa402b50dca62e364c5f2aaa231115a1bedd90ce0a5d7e64cea118ca23fea203b
0x6e5a998b51b39933d2b784b6d15cdefc25bb6705e8692ac74cc635488b28c51a


'0x6e5a998b51b39933d2b784b6d15cdefc25bb6705e8692ac74cc635488b28c51a'

In [43]:
#Check ETH Balance
w3.eth.getBalance("0x263Dafb59dBaC0b39BB2FFF541D05035650ec9A4")


100000000000000001000